## DNA

#### Set-up and initialization

In [1]:
import os
import random
import urllib

import numpy as np

In [2]:
if ((not os.path.isfile(os.path.join('temp', 'hg38.fa.align'))) and
    (not os.path.isfile(os.path.join('temp', 'hg38.align.lines')))):
    url = 'http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.align.gz'
    urllib.request.urlretrieve(url, os.path.join('temp', url.split('/')[-1]))

    import gzip
    import shutil
    with gzip.open(os.path.join('temp', url.split('/')[-1]), 'rb') as f_in:
        with open(os.path.join('temp', 'hg38.fa.align'), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [3]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

count = 0
samp1 = ''
samp2 = ''
first = True
if not os.path.isfile(os.path.join('temp', 'hg38.align.lines')):
    with open(os.path.join('temp', 'hg38.align.lines'), 'w') as f_out:
        with open(os.path.join('temp', 'hg38.fa.align'), 'r') as f_in:
            for line in f_in:
                line_s = line[:-1].split()
                if line_s and line_s[0] in {'Matrix', 'Transitions', 'Gap_init'}:
                    continue
                if len(line_s) > 10 and (line_s[0].isdigit() and is_number(line_s[1]) and
                                         is_number(line_s[2]) and is_number(line_s[3]) and
                                         line_s[-1].isdigit() and line_s[5].isdigit() and
                                         line_s[6].isdigit()):
                    if count:
                        f_out.write(f'{samp1},{samp2}\n')
                    count += 1
                    samp1 = ''
                    samp2 = ''
                    first = True

                elif line:
                    line_s = line[16:].split()
                    if line_s and line_s[0].isdigit() and line_s[2].isdigit():
                        if first:
                            samp1 += line_s[1].strip()
                        else:
                            samp2 += line_s[1].strip()
                        first = not first

        f_out.write(f'{samp1},{samp2}\n')
else:
    with open(os.path.join('temp', 'hg38.align.lines')) as f_count:
        for line in f_count:
            count += 1
count

7203858

#### Sampling and output

In [4]:
np.random.seed(44674)
sel_ct = 2400
selections_ordered = list(np.random.choice(count, sel_ct+400, replace=False))
selections = set(selections_ordered)

seqs = {}
line_no = 0
with open(os.path.join('temp', 'hg38.align.lines'), 'r') as f_in:
    for line in f_in:
        if line_no in selections:
            seqs[line_no] = line.strip()
        line_no += 1

In [5]:
out_ct = 0
sequences = set()

with open('../hg38.csv', 'w') as f_out:
    f_out.write('seq1,seq2\n')
    i = 0
    while out_ct < sel_ct:
        proposed = seqs[selections_ordered[i]].replace('-', '')
        if proposed not in sequences:
            sequences.add(proposed)
            f_out.write(f'{proposed}\n')
            out_ct += 1
        i += 1

In [6]:
random.seed(23115)
out_ct = 0
sequences = []

for i in range(len(selections_ordered)):
    ss1,ss2 = seqs[selections_ordered[i]].split(',')

    start = 0
    if len(ss1)>12:
        start = random.randint(0, len(ss1)-12)
    ss1 = ss1[start:start+12].replace('-', '')
    ss2 = ss2[start:start+12].replace('-', '')

    if len(ss1) < 7 or len(ss2) < 7 or out_ct >= sel_ct or (ss1,ss2) in sequences:
        continue
    else:
        sequences.append((ss1,ss2))
        out_ct += 1

random.shuffle(sequences)
        
with open('../hg38_maxlen12.csv', 'w') as f_out:
    f_out.write('seq1,seq2\n')
    for ss1,ss2 in sequences:
        f_out.write(f'{ss1},{ss2}\n')